In [ ]:
!pip install crewai langchain-openai


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 1.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 285.5/285.5 kB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 75.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.3/135.3 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 72.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 73.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 7.4 MB/s eta 0:00

In [11]:
from crewai import Agent, Task, Crew, Process
from langchain_openai import ChatOpenAI
import os

# === Setup ===
os.environ["OPENAI_API_KEY"] = "..."

# Use GPT-4 or GPT-3.5 as your LLM
llm = ChatOpenAI(model="gpt-4", temperature=0.3)

# === Agents ===
chief_agent = Agent(
    role="Chief Research Agent",
    goal="Oversee the research project and coordinate team tasks effectively.",
    backstory="A seasoned research lead, skilled at breaking down topics and delegating tasks.",
    llm=llm,
    allow_delegation=True
)

literature_agent = Agent(
    role="Literature Specialist",
    goal="Gather relevant and high-quality academic content related to assigned subtopics.",
    backstory="An academic researcher with expertise in scientific literature mining.",
    llm=llm
)

summary_agent = Agent(
    role="Summary Specialist",
    goal="Summarize findings clearly and concisely, focusing on methodology and results.",
    backstory="An analytical thinker and skilled communicator with summarization expertise.",
    llm=llm
)

critic_agent = Agent(
    role="Critique Specialist",
    goal="Critically evaluate research findings and identify strengths and limitations.",
    backstory="A peer reviewer with a keen eye for research rigor and scientific soundness.",
    llm=llm
)

writer_agent = Agent(
    role="Writer",
    goal="Write a final, well-structured, and clear report using all findings.",
    backstory="A science writer experienced in synthesizing research into readable content.",
    llm=llm
)

# === Tasks ===

# Chief Agent decomposes topic
chief_task = Task(
    description="""
    Break the research topic 'Self-supervised learning in medical imaging' into 3 key subtopics.
    Assign each to the Literature Specialist, who will then trigger summaries and critiques.
    Oversee the full workflow and ensure all findings are passed to the Writer.
    """,
    expected_output="Structured subtopics with guidance for literature, summary, and critique.",
    agent=chief_agent
)

literature_task = Task(
    description="Search for top 2-3 recent papers on a given subtopic using academic search tools.",
    expected_output="List of paper titles, brief descriptions, and source links.",
    agent=literature_agent
)

summary_task = Task(
    description="Summarize each paper found in bullet points, highlighting goals, methods, and findings.",
    expected_output="Summaries for each paper (1-2 paragraphs each).",
    agent=summary_agent
)

critique_task = Task(
    description="Critically assess each summary. Identify research gaps, assumptions, and methodological issues.",
    expected_output="1 paragraph of critical analysis per paper.",
    agent=critic_agent
)

writer_task = Task(
    description="Combine all outputs (summaries and critiques) into a polished 700-word research report.",
    expected_output="Structured report with introduction, body, and conclusion.",
    agent=writer_agent
)

# === Define Hierarchical Process ===
crew = Crew(
    agents=[
        literature_agent,
        summary_agent,
        critic_agent,
        writer_agent
    ],
    tasks=[
        chief_task,
        literature_task,
        summary_task,
        critique_task,
        writer_task
    ],
    process=Process.hierarchical,  # ⬅️ Hierarchical orchestration
    manager_agent=chief_agent,
    verbose=True
)

# === Run it! ===
result = crew.kickoff()
print("\n\n📝 Final Report Output:\n", result)


╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 741a331f-cb2f-4d15-a75a-52e3e841c19e                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: 635b4410-bd14-4e74-901c-041f43b2d644
       Status: Executing Task...

🚀 Crew: crew
└── 📋 Task: 635b4410-bd14-4e74-901c-041f43b2d644
       Status: Executing Task...
    └── 🤖 Agent: Chief Research Agent
            Status: In Progress

# Agent: Chief Research Agent
## Task: 
    Break the research topic 'Self-supervised learning in medical imaging' into 3 key subtopics.
    Assign each to the Literature Specialist, who will then trigger summaries and critiques.
    Oversee the full workflow and ensure all findings are passed to the Writer.
    


🤖 Agent: Chief Research Agent
    Status: In Progress

...

🚀 Crew: crew
├── 📋 Task: 635b4410-bd14-4e74-901c-041f43b2d644
│      Assigned to: Chief Research Agent
│      Status: ✅ Completed
│   ├── 🤖 Agent: Chief Research Agent
│   │       Status: In Progress
│   ├── 🤖 Agent: Chief Research Agent
│   │       Status: ✅ Completed
│   ├── 🤖 Agent: Chief Research Agent
│   │       Status: ✅ Completed
│   ├── 🤖 Agent: Chief Research Agent
│   │       Status: ✅ Completed
│   └── 🤖 Agent: Chief Research Agent
│           Status: ✅ Completed
├── 📋 Task: 1da81467-09be-45a5-b708-5e1fe8c3040e
│      Assigned to: Chief Research Agent
│      Status: ✅ Completed
│   ├── 🤖 Agent: Chief Research Agent
│   │       Status: In Progress
│   └── 🤖 Agent: Chief Research Agent
│           Status: ✅ Completed
├── 📋 Task: c10baf1c-14ed-4f89-98a5-930330348a0d
│      Assigned to: Chief Research Agent
│      Status: ✅ Completed
│   ├── 🤖 Agent: Chief Research Agent
│   │       Status: In Progress
│   └── 🤖 Agent: Chief Research Agent
│           Status: ✅ Completed
├── 📋 Task: 55d40e36-0d94-4a96-9941-7bee78870518
│      Assigned to: Chief Research Agent
│      Status: ✅ Completed
│   ├── 🤖 Agent: Chief Research Agent
│   │       Status: In Progress
│   └── 🤖 Agent: Chief Research Agent
│           Status: ✅ Completed
└── 📋 Task: 5aa08245-4080-4c87-ae9d-6a3d4726ea86
       Status: Executing Task...
    ├── 🤖 Agent: Chief Research Agent
    │       Status: In Progress
    └── 🤖 Agent: Chief Research Agent
            Status: ✅ Completed

🚀 Crew: crew
├── 📋 Task: 635b4410-bd14-4e74-901c-041f43b2d644
│      Assigned to: Chief Research Agent
│      Status: ✅ Completed
│   ├── 🤖 Agent: Chief Research Agent
│   │       Status: In Progress
│   ├── 🤖 Agent: Chief Research Agent
│   │       Status: ✅ Completed
│   ├── 🤖 Agent: Chief Research Agent
│   │       Status: ✅ Completed
│   ├── 🤖 Agent: Chief Research Agent
│   │       Status: ✅ Completed
│   └── 🤖 Agent: Chief Research Agent
│           Status: ✅ Completed
├── 📋 Task: 1da81467-09be-45a5-b708-5e1fe8c3040e
│      Assigned to: Chief Research Agent
│      Status: ✅ Completed
│   ├── 🤖 Agent: Chief Research Agent
│   │       Status: In Progress
│   └── 🤖 Agent: Chief Research Agent
│           Status: ✅ Completed
├── 📋 Task: c10baf1c-14ed-4f89-98a5-930330348a0d
│      Assigned to: Chief Research Agent
│      Status: ✅ Completed
│   ├── 🤖 Agent: Chief Research Agent
│   │       Status: In Progress
│   └── 🤖 Agent: Chief Research Agent
│           Status: ✅ Completed
├── 📋 Task: 55d40e36-0d94-4a96-9941-7bee78870518
│      Assigned to: Chief Research Agent
│      Status: ✅ Completed
│   ├── 🤖 Agent: Chief Research Agent
│   │       Status: In Progress
│   └── 🤖 Agent: Chief Research Agent
│           Status: ✅ Completed
└── 📋 Task: 5aa08245-4080-4c87-ae9d-6a3d4726ea86
       Assigned to: Chief Research Agent
       Status: ✅ Completed
    ├── 🤖 Agent: Chief Research Agent
    │       Status: In Progress
    └── 🤖 Agent: Chief Research Agent
            Status: ✅ Completed

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 5aa08245-4080-4c87-ae9d-6a3d4726ea86                                                                     │
│  Agent: Chief Research Agent                                                                                    │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 741a331f-cb2f-4d15-a75a-52e3e841c19e                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯



📝 Final Report Output:
 Title: Self-Supervised Learning in Medical Imaging: A Comprehensive Analysis

Introduction:

Self-supervised learning, a subset of machine learning, has recently emerged as a promising approach in the field of medical imaging. Unlike supervised learning that requires labeled data, self-supervised learning leverages unlabeled data, making it a cost-effective and efficient solution for medical imaging. This report explores the application, benefits, and challenges of self-supervised learning in medical imaging.

Body:

1. Application of Self-Supervised Learning in Medical Imaging:

Self-supervised learning has found extensive application in medical imaging, particularly in areas where labeled data is scarce or expensive to obtain. For instance, it is used in the detection and diagnosis of diseases such as cancer, Alzheimer's, and cardiovascular diseases. By learning from the inherent patterns in the unlabeled medical images, self-supervised algorithms can identi